In [2]:
import cudf
print('RAPIDS cuDF version',cudf.__version__)

train = cudf.read_csv('data/training_set.csv')
test = cudf.read_csv('data/public_testset.csv')

train_pairs = train[["UserId", "ItemId"]].rename(columns={"UserId": "session", "ItemId": "aid"})
# # train_pairs = cudf.concat([train, test])[['session', 'aid']]
# # del train, test

train_pairs['aid_next'] = train_pairs.groupby('session').aid.shift(-1)
train_pairs = train_pairs[['aid', 'aid_next']].dropna().reset_index(drop=True)

cardinality_aids = max(train_pairs['aid'].max(), train_pairs['aid_next'].max())
print('Cardinality of items is',cardinality_aids)

RAPIDS cuDF version 24.10.01
Cardinality of items is zzyfHckCgU


In [1]:
from merlin.loader.torch import Loader 

train_pairs.to_pandas().to_parquet('train_pairs.parquet') # TRAIN WITH ALL DATA
train_pairs[-10_000_000:].to_pandas().to_parquet('valid_pairs.parquet')

# from merlin.loader.torch import Loader| 
from merlin.io import Dataset

train_ds = Dataset('train_pairs.parquet')
train_dl_merlin = Loader(train_ds, 65536, True)

/opt/conda/envs/rapids-24.10/lib/python3.10/site-packages/merlin/dtypes/mappings/tf.py:52: UserWarning: Tensorflow dtype mappings did not load successfully due to an error: No module named 'tensorflow'
  warn(f"Tensorflow dtype mappings did not load successfully due to an error: {exc.msg}")
/opt/conda/envs/rapids-24.10/lib/python3.10/site-packages/merlin/dtypes/mappings/triton.py:53: UserWarning: Triton dtype mappings did not load successfully due to an error: No module named 'tritonclient'
  warn(f"Triton dtype mappings did not load successfully due to an error: {exc.msg}")
/opt/conda/envs/rapids-24.10/lib/python3.10/site-packages/merlin/loader/torch.py:24: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  np.bool: torch.bool,


AttributeError: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [2]:
from cudf.core.column import build_column